# Install dependencies if not installed


( !pip install selenium webdriver-manager pandas )


In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# Configure Selenium WebDriver


In [ ]:
options = Options()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")


In [ ]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# List of Jektis Travel hotel pages to scrape


In [ ]:
urls = {
    "Sousse": "https://www.jektistravel.com/hotels-en-tunisie/Sousse",
    "Djerba": "https://www.jektistravel.com/hotels-en-tunisie/Djerba",
    "Monastir": "https://www.jektistravel.com/hotels-en-tunisie/Monastir",
    "Hammamet": "https://www.jektistravel.com/hotels-en-tunisie/Hammamet"
}

# Function to scrape hotels from a given URL


In [ ]:
def scrape_hotels(url, city):
    driver.get(url)
    time.sleep(5)  # Allow page to load
    
    hotels = []
    hotel_cards = driver.find_elements(By.CLASS_NAME, "block-hotel-ville")
    
    for card in hotel_cards:
        try:
            name = card.find_element(By.CLASS_NAME, "name-voyage").text.strip()
            location = card.find_element(By.CLASS_NAME, "localisation-voyage").text.strip()
            stars = len(card.find_elements(By.CSS_SELECTOR, ".booking-item-rating-stars .fa-star"))
            
            try:
                label = card.find_element(By.CLASS_NAME, "a-partir-de").text.strip()
                amount = card.find_element(By.CLASS_NAME, "price-voyage").find_element(By.TAG_NAME, "span").text.strip()
                price = f"{label} {amount} TND"
            except:
                price = "No Price"
            
            hotels.append([name, location, stars, price, city])
        
        except Exception as e:
            print(f"Error extracting data for a hotel in {city}: {e}")
            continue
    
    return hotels


# Scrape all cities


In [ ]:
all_hotels = []
for city, url in urls.items():
    print(f"Scraping hotels in {city}...")
    city_hotels = scrape_hotels(url, city)
    all_hotels.extend(city_hotels)

# Close WebDriver


In [ ]:
driver.quit()


# Save data to CSV


In [ ]:
df = pd.DataFrame(all_hotels, columns=["Hotel Name", "Location", "Stars", "Price", "City"])
df.to_csv("jektis_hotels.csv", index=False, encoding="utf-8")

In [ ]:
print(f" Scraping completed! {len(all_hotels)} hotels saved in 'jektis_hotels.csv'.")
